# Model Selection Using Cross-Validation for the Advanced Fibrosis Diagnosis Classifier

In this activity, we are going to improve our classifier for the hepatitis C dataset by using cross-validation for model selection and hyperparameter selection.

### 1. Import the required packages. Load the dataset from the data subfolder of the Chapter04 folder from GitHub using X = pd.read_csv('../data/HCV_feats.csv'), y = pd.read_csv('../data/HCV_target.csv').

In [1]:
import pandas as pd 
import numpy as np 
from tensorflow import random
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

SEED = 2

Using TensorFlow backend.


In [2]:
X = pd.read_csv('../data/HCV_feats.csv')
y = pd.read_csv('../data/HCV_target.csv')

In [3]:
y

,AdvancedFibrosis
0,0
1,0
2,1
3,1
4,0
...,...
1380,1
1381,0
1382,0
1383,1


In [4]:
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X

,Age,Gender,BMI,Fever,Nausea/Vomting,Headache,Diarrhea,Fatigue & generalized bone ache,Jaundice,Epigastric pain,...,ALT 24,ALT 36,ALT 48,ALT after 24 w,RNA Base,RNA 4,RNA 12,RNA EOT,RNA EF,Baseline histological Grading
0,1.102814,-0.979276,1.568525,0.969420,-1.005067,-0.992089,-1.005067,1.002168,0.997836,0.992089,...,-0.103412,-2.960181,-2.999471,-4.021808,0.181960,0.092882,-0.001962,-1.087692,-1.088823,0.805050
1,-0.036355,-0.979276,0.096039,-1.031544,0.994959,1.007974,-1.005067,1.002168,0.997836,-1.007974,...,1.118124,-0.989700,1.501857,1.493666,-1.555454,-0.171903,1.221053,0.185824,-0.972681,-1.432396
2,1.216730,-0.979276,1.077696,0.969420,0.994959,1.007974,0.994959,-0.997836,-1.002168,-1.007974,...,1.232643,-2.960181,-2.999471,-4.021808,-0.055972,0.166905,-1.012273,1.695069,0.999427,-1.432396
3,0.305396,1.021163,1.077696,-1.031544,0.994959,-0.992089,0.994959,-0.997836,0.997836,-1.007974,...,0.163799,-1.330745,-0.252898,-0.061981,1.274675,-0.416795,1.040971,1.727277,1.087139,0.059234
4,1.444564,-0.979276,0.832282,-1.031544,-1.005067,1.007974,-1.005067,1.002168,0.997836,0.992089,...,1.385335,0.412373,0.243011,-0.486248,0.196318,0.380636,12.069419,0.193923,-0.181303,0.307840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380,-0.264188,-0.979276,0.096039,-1.031544,0.994959,1.007974,0.994959,-0.997836,-1.002168,-1.007974,...,1.652546,-0.762337,-1.511744,1.635088,-0.574200,-1.504642,-1.012273,-1.087692,-1.088823,1.302260
1381,0.988897,-0.979276,1.323110,-1.031544,0.994959,1.007974,-1.005067,-0.997836,-1.002168,-1.007974,...,0.927259,0.526054,-0.748807,1.069398,-0.309697,-1.236759,0.366648,-0.809510,-0.205921,0.059234
1382,-0.492022,-0.979276,-0.640203,0.969420,0.994959,-0.992089,-1.005067,-0.997836,0.997836,-1.007974,...,1.232643,0.147116,-1.702479,-1.334782,0.061369,-0.077694,1.813706,0.211971,-0.489235,-0.935186
1383,0.647146,-0.979276,0.096039,0.969420,-1.005067,-0.992089,0.994959,1.002168,0.997836,-1.007974,...,-1.401293,-1.330745,-0.100311,1.352243,-1.274928,-1.448806,0.795717,-1.078409,1.512293,1.302260


In [5]:
print(X.shape)
print(y.shape)

(1385, 28)
(1385, 1)


### 2. Define three functions, each returning a different Keras model. The first Keras model will be a deep neural network with three hidden layers all of size 4 and ReLU activation functions. The second Keras model will be a deep neural network with two hidden layers, the first layer of size 4 and the second later of size 2, and ReLU activation functions. The third Keras model will be a deep neural network with two hidden layers, both of size 8, and a ReLU activation function. Use the following values for the hyperparameters: optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']

In [6]:
def build_model_1(activation='relu', optimizer='adam', loss='binary_crossentropy'):
    model = Sequential()
    model.add(Dense(4, input_dim=X.shape[1], activation=activation))
    model.add(Dense(4, activation=activation))
    model.add(Dense(4, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'],)
    return model

def build_model_2(activation='relu', optimizer='adam', loss='binary_crossentropy'):
    model = Sequential()
    model.add(Dense(4, input_dim=X.shape[1], activation=activation))
    model.add(Dense(2, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'],)
    return model

def build_model_3(activation='relu', optimizer='adam', loss='binary_crossentropy'):
    model = Sequential()
    model.add(Dense(8, input_dim=X.shape[1], activation=activation))
    model.add(Dense(8, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'],)
    return model

### 3. Write the code that will loop over the three models and perform 5-fold cross-validation on each of them (use epochs=100, batch_size=20, and shuffle=False in this step). Store all the cross-validation scores in a list and print the results. Which model results in the best accuracy?

In [73]:
models = [build_model_1, build_model_2, build_model_3]

params_1 = {
    'epochs': 100,
    'batch_size': 20,
    'shuffle': False,
    'verbose': 1,
}

results_1 = []

for m in range(len(models)):
    # set seed
    np.random.seed(SEED)
    random.set_seed(SEED)
    # get classifier
    classifier = KerasClassifier(build_fn=models[m], **params_1)
    # perform cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    result = cross_val_score(classifier, X, y, cv=cv, verbose=1)
    # append results to list
    results_1.append(result)


1108/1108 [==============================] - 0s 165us/step - loss: 0.6631 - accuracy: 0.6498
Epoch 17/100
1108/1108 [==============================] - 0s 134us/step - loss: 0.6607 - accuracy: 0.6498
Epoch 18/100
1108/1108 [==============================] - 0s 145us/step - loss: 0.6582 - accuracy: 0.6480
Epoch 19/100
1108/1108 [==============================] - 0s 154us/step - loss: 0.6558 - accuracy: 0.6507
Epoch 20/100
1108/1108 [==============================] - 0s 136us/step - loss: 0.6530 - accuracy: 0.6552
Epoch 21/100
1108/1108 [==============================] - 0s 137us/step - loss: 0.6502 - accuracy: 0.6588
Epoch 22/100
1108/1108 [==============================] - 0s 143us/step - loss: 0.6475 - accuracy: 0.6652
Epoch 23/100
1108/1108 [==============================] - 0s 166us/step - loss: 0.6448 - accuracy: 0.6661
Epoch 24/100
1108/1108 [==============================] - 0s 117us/step - loss: 0.6421 - accuracy: 0.6706
Epoch 25/100
1108/1108 [==============================] - 

In [75]:
# print results
for m in range(len(models)):
    print(f'Model_{m+1} test accuracy: {results_1[m].mean()}')

Model_1 test accuracy: 0.4945848286151886
Model_2 test accuracy: 0.5487364649772644
Model_3 test accuracy: 0.49819493293762207


Result was:

Model_2 test accuracy: 0.5487364649772644

Select Model_2

### 4. Write the code that uses the epochs = [100, 200] and batches = [10, 20] values for epochs and batch_size. Perform 5-fold cross-validation for each possible pair on the Keras model that resulted in the best accuracy from step 3. Store all the cross-validation scores in a list and print the results. Which epochs and batch_size pair results in the best accuracy?

In [7]:
epochs = [100, 200]
batches = [10, 20]

params_2 = {
    'shuffle': False,
    'verbose': 1,
}

results_2 = []

for e in range(len(epochs)):
    for b in range(len(batches)):
        # set seed
        np.random.seed(SEED)
        random.set_seed(SEED)
        # get classifier
        classifier = KerasClassifier(
            build_fn=build_model_2, 
            epochs=epochs[e], 
            batch_size=batches[b], 
            **params_2
        )
        # perform cross-validation
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
        result = cross_val_score(classifier, X, y, cv=cv, verbose=1)
        # append results to list
        results_2.append(result)

16/200
1108/1108 [==============================] - 0s 136us/step - loss: 0.6769 - accuracy: 0.5830
Epoch 17/200
1108/1108 [==============================] - 0s 112us/step - loss: 0.6750 - accuracy: 0.5821
Epoch 18/200
1108/1108 [==============================] - 0s 116us/step - loss: 0.6733 - accuracy: 0.5821
Epoch 19/200
1108/1108 [==============================] - 0s 112us/step - loss: 0.6716 - accuracy: 0.5857
Epoch 20/200
1108/1108 [==============================] - 0s 105us/step - loss: 0.6700 - accuracy: 0.5939
Epoch 21/200
1108/1108 [==============================] - 0s 137us/step - loss: 0.6683 - accuracy: 0.5948
Epoch 22/200
1108/1108 [==============================] - 0s 101us/step - loss: 0.6667 - accuracy: 0.5984
Epoch 23/200
1108/1108 [==============================] - 0s 145us/step - loss: 0.6654 - accuracy: 0.5993
Epoch 24/200
1108/1108 [==============================] - 0s 118us/step - loss: 0.6642 - accuracy: 0.6038
Epoch 25/200
1108/1108 [============================

In [8]:
# print results
c = 0
for e in range(len(epochs)):
    for b in range(len(batches)):
        print(f'Epoch: {epochs[e]} Batch_Size:{batches[b]} test accuracy: {results_2[c].mean()}')
        c += 1

Epoch: 100 Batch_Size:10 test accuracy: 0.542960274219513
Epoch: 100 Batch_Size:20 test accuracy: 0.5415162324905396
Epoch: 200 Batch_Size:10 test accuracy: 0.535740077495575
Epoch: 200 Batch_Size:20 test accuracy: 0.5407942295074463


### 5. Write the code that uses the optimizers = ['rmsprop', 'adam','sgd'] and activations = ['relu', 'tanh'] values for optimizer and activation. Perform 5-fold cross-validation for each possible pair on the Keras model that resulted in the best accuracy from step 3. Use the batch_size and epochs values that resulted in the best accuracy from step 4. Store all the cross-validation scores in a list and print the results. Which optimizer and activation pair results in the best accuracy?

In [ ]:
optimizers = ['rmsprop', 'adam','sgd']
activations = ['relu', 'tanh']

params_3 = {
    'epochs': 100,
    'batch_size': 10,
    'shuffle': False,
    'verbose': 1,
}

results_3 = []

for o in range(len(optimizers)):
    for a in range(len(activations)):
        # set seed
        np.random.seed(SEED)
        random.set_seed(SEED)
        # get classifier
        classifier = KerasClassifier(
            build_fn=build_model_2,
            optimizer=optimizers[o],
            activation=activations[a],
            **params_3
        )
        # perform cross-validation
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
        result = cross_val_score(classifier, X, y, cv=cv, verbose=1)
        # append results to list
        results_3.append(result)

In [ ]:
# print results
c = 0
for o in range(len(optimizers)):
    for a in range(len(activations)):
        print(f'Optimizer: {optimizers[o]} Activation:{activations[a]} test accuracy: {results_3[c].mean()}')
        c += 1